This example shows how to use hydrolinking methods to address a point to the NHDPlusV2.1 Medium Resolution.

For this example we use a point near the Red Cedar River in East Lansing, MI.

##### First set required parameters.
The input parameters are the same for NHDPlusV2.1 and NHD High Resolution hydrolinking methods.

Required parameters include:
    *input_identifier: str, user supplied identifier
    *input_lat: float, latitude of the point to be hydrolinked
    *input_lon: float, longitude of the point to be hydrolinked
    
Other optional parameters include:
    *input_crs: int, coordinate reference system, default is 4269 (NAD83)
    *stream_name: name of the stream a point is intended to be linked
    *buffer_m = search distance in meters, default is 1000, max is 2000

In [1]:
#identifier -> number or string
input_identifier = 1
#latitude
input_lat = 42.7284
#longitude
input_lon = -84.5026
#coordinate system
input_crs = 4269  
#stream name
stream_name = 'The Red Cedar River'
#buffer
buffer_m = 2000

The next cell runs three steps. First the hydrolink module for nhdplusV2.1 is imported.  Next a Python object is initiated and finally the code attempts to set the coordinate system to NAD83 (crs = 4269).

Note: It is recommended to use crs 4269, although this code attempts to accomidate other common coordinate systems by using some simple logic.  This has not been tested a lot and may cause the code to bypass hydrolinking if conversion fails. If this happens a message will be generated for the user.

In [3]:
#Import module from local folder hydrolink
from hydrolink import nhd_mr
import warnings; warnings.simplefilter('ignore')

#initiate a Python object
hydrolink = nhd_mr.MedResPoint(input_identifier, input_lat, input_lon, water_name=stream_name, input_crs=input_crs, buffer_m=buffer_m)

#We then try setting the coordinate system to NAD83
hydrolink.crs_to_4269()

In [6]:
#builds query against EPA web services
hydrolink.build_nhd_query(service='network_flow')

#executes query and measures distances from point to each line and their nodes
hydrolink.find_closest_reaches(n=6)

#stream name match for each of the reaches
hydrolink.water_name_match()

#show result, there were only 3 reaches within 2000 meters of this point
hydrolink.reach_df

,GNIS_NAME,LENGTHKM,PERMANENT_IDENTIFIER,REACHCODE,COMID,closest_node_m,snap_m,snap_xy,closest,name_check,name_check_txt
0,Red Cedar River,8.421667,12242482,04050004000126,12242482,1987.423043,44.332661,POINT (-84.50243779700608 42.72877868601643),1,0.882353,most likely name match based on fuzzy match
1,None,1.983866,12241744,04050004000960,12241744,1987.423043,1837.753163,POINT (-84.50737777864552 42.71232846672766),2,0.000000,"no name match, water name and or gnis name not..."
2,Red Cedar River,0.790011,12241734,04050004000125,12241734,1987.423043,1987.423043,POINT (-84.52110377914246 42.71680106720647),3,0.882353,most likely name match based on fuzzy match


In the next cell, a selection of one reach is made using the following criteria.  This reach selection identifies the "best reach".  In other words, of the reaches returned in the query this step selects the reach most likely associated with the input data point.
 
        * First check for exact stream name matches (GNIS_NAME == stream_name)
           * If number of reaches with exact name match equals 1 that is the reach to recommend
           * If number of reaches with exact name match > 1 take the one that is closest to the point
           * If more than one reach has exact name match and are equally close to the point grab the first but note that                               multiple reaches so that we can recommend taking a closer look
        * If no reaches have an exact name match and the user supplied name does not refer to a tributary then check for fuzzy name matches over 0.75 cutoff.  
            * If number of reaches with fuzzy name match equals 1 that is the reach to recommend
            * If number of reaches with fuzzy name match > 1 take the one that is closest to the point
                * If more than one reach has fuzzy name match and are equally close to the point grab the first one but note                           that multiple reaches so that we can recommend taking a closer look
        * If fuzzy match < 0.75 just take closest reach.
        

In [7]:
#Return reach most likely associated with point
hydrolink.select_best_reach()

#show results
hydrolink.best_reach

{'input_id': 1,
 'water_name': 'The Red Cedar River',
 'water_name_ref': 'the red cedar river',
 'GNIS_NAME': 'Red Cedar River',
 'LENGTHKM': 8.42166733289811,
 'PERMANENT_IDENTIFIER': '12242482',
 'COMID': 12242482,
 'REACHCODE': '04050004000126',
 'snap_xy': <shapely.geometry.point.Point at 0x178d67c1dc8>,
 'snap_m': 44.332660593521666,
 'closest_node_m': 1987.4230429025345,
 'closest': 1,
 'name_check': 0.8823529411764706,
 'mult_reach_ct': 1,
 'name_check_txt': 'most likely name match based on fuzzy match',
 'mr_meas': None,
 'message': ''}

The following cell finalizes point addressing by retrieving the measure along the "best reach" where the point should be associated.  This method takes a striaght line snap approach.  The measure value is measured as a percent of the way "up" a reach(0-100) where 0 is at the most downstream node of the reach and 100 is the most upstream node of the reach. Note that all reachcodes start at 0 and end at 100, but there may be multiple flowlines that make up a single reach.

In [8]:
hydrolink.get_hl_measure()

hydrolink.best_reach['mr_meas']

30.44817

There are a few options for writing data to CSV files.  Both are set up with intentions of batch hydrolinking (multiple points -> see batch example notebook). 

Both methods accept an optional parameter for 'outfile_name'. 
    * default for write_reach_options() is 'mr_hydrolink_output.csv'
    * default for write_best() is 'mr__hydrolink_output.csv'

In [9]:
#Creates a CSV file of all reach options.  This can be used for QAQC procedures.
hydrolink.write_reach_options()

#Creates a CSV file with one reach per point.  
hydrolink.write_best()

In [10]:
#View final data for best reach
import pandas as pd
pd.DataFrame([hydrolink.best_reach])

,input_id,water_name,water_name_ref,GNIS_NAME,LENGTHKM,PERMANENT_IDENTIFIER,COMID,REACHCODE,snap_xy,snap_m,closest_node_m,closest,name_check,mult_reach_ct,name_check_txt,mr_meas,message
0,1,The Red Cedar River,the red cedar river,Red Cedar River,8.421667,12242482,12242482,04050004000126,POINT (-84.50243779700608 42.72877868601643),44.332661,1987.423043,1,0.882353,1,most likely name match based on fuzzy match,30.44817,
